# Setup
## Import Libraries

In [1]:
%pylab inline
import scipy as sp
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm, Normalize
from mpl_toolkits import mplot3d
from sklearn.metrics import mean_squared_error
from statannot import add_stat_annotation

import warnings
import itertools


Populating the interactive namespace from numpy and matplotlib


# K2: singles and pair effect experiment

## Setup


### Create lists of sample types for later use  

In [2]:
isolates=['BI', 'CB', 'CF', 'EA', 'EC1', 'EC10', 'EC11', 'EC12','EC14', 
          'EC16','EC17', 'EC18', 'EC9', 'Ecoli', 'Gh11','Gh12','Gh16',
          'Gh22', 'Gh24','Gh28', 'Gh29', 'Gh30', 'Gh35', 'Gh36','Gh38',
          'Gh40', 'Gh43', 'Gh44', 'Gh50', 'Gh54', 'Gh59', 'Gh60','Gh61',
          'Gh62', 'Gh67', 'Gh68', 'Gh9','KA', 'LA', 'OGI19-L','OGI33-L',
          'Og122', 'Og84', 'Og85', 'Og87', 'Og90','Og92', 'PAg1','PAg3', 
          'PAl', 'PH', 'PK', 'PP', 'ParPR2', 'RP1', 'RP1_A','RP1_B', 
          'RP2', 'SF1', 'SF2', 'Ut12', 'Ut14', 'Ut22']
monos=  ['Mono1','Mono2','Mono']
blanks= ['Blank1','Blank2']

timepoints=['t0','t1','t2','t3']


### Load and orginze data

In [3]:
warnings.filterwarnings("ignore")

#Pair chips with their data files
data_files=dict(zip(range(1,13),["EC_A","EC_B","EA_A","EA_B","RP1_A","RP1_B","BI_A","BI_B",
                                 "CF_A","CF_B","PAg_A","PAg_B"]))

#Load data from all chips into dictionary
chips_dict={}
for chip in range(1,13):
    chips_dict[data_files[chip]]=pd.read_csv('Data/kChip_data/k2/chip'+str(chip)+'.csv')
    
    area_mode=chips_dict[data_files[chip]]['t0_Area'].mode()[0]
    margin=area_mode*0.3
    chips_dict[data_files[chip]]=chips_dict[data_files[chip]][(chips_dict[data_files[chip]].t0_Area>(area_mode-margin)) &
                           (chips_dict[data_files[chip]].t0_Area<(area_mode+margin)) & (chips_dict[data_files[chip]].Total==2)]
    chips_dict[data_files[chip]]['chip']=chr(ord('@')+(chip%2+1))
    
#Merge replicate chips
chips_dict2={}
for focal in ["EC","EA","RP1","BI","CF","PAg"]:
    chips_dict2[focal]=chips_dict[focal+'_A'].append(chips_dict[focal+'_B'])

chips_dict=chips_dict2

"""Subtract t0 value for each well individually (so we are measuring added biomass)"""

n_chips_dict={}
for chip in chips_dict:
    df=chips_dict[chip].copy()

    #normalize each well to its own starting value
    df.t1=df.t1-df.t0
    df.t2=df.t2-df.t0
    df.t3=df.t3-df.t0
    df.t0=df.t0-df.t0
    
    #ensure no negative values or zero to avoid inf values later on
    df[timepoints]=df[timepoints].clip(lower=1)
    
    n_chips_dict[chip]=df

### Load growth curve data

In [4]:
#dictionary to match samples to identifiers
phylo=pd.read_excel('Data/Strains.xlsx')
r_phylo_dict=dict(zip(phylo['Identifier'],phylo['Sample Name']))


gc_data=(pd.read_csv('Data/Isolate_profiling/gc_data.csv')
         .replace(r_phylo_dict)
         .replace({'EColi':'Ecoli'})
         .drop_duplicates(subset='sample',)
         .set_index('sample'))

max_od_dict=dict(zip(gc_data.index,gc_data.max_od))

In [5]:
def modified_S (row):
    if row["mono_s1_effect"]>0 and row["mono_s2_effect"]>0:
        row['modified_strongest']=row[["mono_s1_effect","mono_s2_effect"]].max()
    else: row['modified_strongest']=row[["mono_s1_effect","mono_s2_effect"]].min()
    return row

def comm_size(row):
    
    #assign community size
    if row['sample1']==row['sample2']: 
        row['comm_size']='single'
    elif row['sample1']=='Mono': 
        row['comm_size']='single'    
    elif row['sample2']=='Mono': 
        row['comm_size']='single'
    else: row['comm_size']='pair'    
    
    return row

## Test on original dataset and compare values

In [6]:
warnings.filterwarnings("ignore")
orngl_effects_dict={}

for chip in list(n_chips_dict.keys()):
    tp='t1'
    data=n_chips_dict[chip].copy()
    focal_mono_df=data[(data['sample1'].isin(monos))&
                           (data['sample2'].isin(monos))][tp]
    focal_mono_median=focal_mono_df.median()
    
    data[['sample1','sample2']]= data[['sample1','sample2']].replace({'RP1_A':'RP1','RP1_B':'RP1',
                                                                      'Blank1':np.nan,'Blank2':np.nan,
                                                                      'Mono1':np.nan,'Mono2':np.nan,
                                                                      'Mono':np.nan})

    data.sample1 = data.sample1.fillna(data.sample2)
    data.sample2 = data.sample2.fillna(data.sample1)
    
    #get data for pair
    effect_data=data.groupby(['sample1', 'sample2'],
                             as_index=False)[tp].median()
    
    effect_data['combined_effect']=log(effect_data[tp]/focal_mono_median)
    
    effect_data['Count']=data.groupby(['sample1', 'sample2'],
                                      as_index=False)[tp].count()[tp]

#     add data for each bug on its own
    mono_t=(effect_data[(effect_data.sample1==effect_data.sample2)|
                        (effect_data.sample1.isin(monos))|
                        (effect_data.sample2.isin(monos))]
          [['sample1','sample2','combined_effect', 'Count']])

    mono_t.sample1=mono_t.sample1.replace(['Mono'], np.nan)
    mono_t.sample1 = mono_t.sample1.fillna(mono_t.sample2)

    mono_t.sample2=mono_t.sample2.replace(['Mono'], np.nan)
    mono_t.sample2 = mono_t.sample2.fillna(mono_t.sample1)

    mono=pd.DataFrame(mono_t.groupby(['sample1','sample2'],
                                    as_index=False)['combined_effect'].mean())
    mono['Count']=mono_t.groupby(['sample1','sample2'],
                                as_index=False)['Count'].sum()['Count']           
          
    mono=mono.rename(columns={'combined_effect':'effect'})
    
    mono['max_od'] = mono.sample1.map(max_od_dict)
    
    monos1=mono.add_prefix('mono_s1_')
    effect_data=effect_data.merge(monos1, left_on='sample1',
                                  right_on='mono_s1_sample1').drop('mono_s1_sample1',axis=1)
    monos2=mono.add_prefix('mono_s2_')
    effect_data=effect_data.merge(monos2, left_on='sample2',
                                  right_on='mono_s2_sample1').drop('mono_s2_sample1',axis=1)

    effect_data['strongest'] = effect_data.apply(lambda x: max(x[["mono_s1_effect","mono_s2_effect"]],
                                                               key=abs), axis=1)
    
    effect_data['mean'] = effect_data[["mono_s1_effect","mono_s2_effect"]].mean(axis=1)
    
    effect_data['additive']=(effect_data["mono_s1_effect"]+
                             effect_data["mono_s2_effect"])
    
    effect_data['OD_W_mean']=(((effect_data["mono_s1_effect"]*effect_data["mono_s1_max_od"])+
                                  (effect_data["mono_s2_effect"]*effect_data["mono_s2_max_od"]))/
                                  (effect_data["mono_s1_max_od"]+effect_data["mono_s2_max_od"]))  
    
    effect_data['focal']=chip
    effect_data=effect_data.apply(modified_S, axis=1)
    effect_data=effect_data.apply(comm_size, axis=1)
        
    effect_data=effect_data[effect_data['Count']>2]
    orngl_effects_dict[chip]=effect_data
    
#merge into single dataframe
original_dataset=orngl_effects_dict['EC'].copy()
for chip in list(orngl_effects_dict.keys())[1:]:
    original_dataset=original_dataset.append(orngl_effects_dict[chip])

    

In [7]:
pairs=original_dataset[original_dataset.comm_size=='pair']#.drop_duplicates()

for m in ['additive','mean', 'OD_W_mean', 'strongest','modified_strongest']:
    
    effects_df2=pairs[[m,'combined_effect']].dropna()
    iqr=sp.stats.iqr(effects_df2.combined_effect)

    print(m,(((mean_squared_error(effects_df2[m],effects_df2['combined_effect']))**0.5)/iqr).round(3))
        


additive 0.663
mean 0.464
OD_W_mean 0.361
strongest 0.263
modified_strongest 0.244


## Bootstrapping

In [8]:
repeats=1001

warnings.filterwarnings("ignore")
k2_effects_dict={}
df_counter=1
for chip in list(n_chips_dict.keys()):
    tp='t1'
    data2=n_chips_dict[chip].copy()
    for i in range(1,repeats):
        data=data2.sample(len(data2), replace=True, random_state=i, ignore_index=True)

        focal_mono_df=data[(data['sample1'].isin(monos))&
                       (data['sample2'].isin(monos))][tp]
        focal_mono_median=focal_mono_df.median()

        data[['sample1','sample2']]= data[['sample1','sample2']].replace({'RP1_A':'RP1','RP1_B':'RP1',
                                                                          'Blank1':np.nan,'Blank2':np.nan,
                                                                          'Mono1':np.nan,'Mono2':np.nan,
                                                                          'Mono':np.nan})

        data.sample1 = data.sample1.fillna(data.sample2)
        data.sample2 = data.sample2.fillna(data.sample1)

        #get data for pair
        effect_data=data.groupby(['sample1', 'sample2'],
                                 as_index=False)[tp].median()

        effect_data['combined_effect']=log(effect_data[tp]/focal_mono_median)

        effect_data['Count']=data.groupby(['sample1', 'sample2'],
                                          as_index=False)[tp].count()[tp]

    #     add data for each bug on its own
        mono_t=(effect_data[(effect_data.sample1==effect_data.sample2)|
                            (effect_data.sample1.isin(monos))|
                            (effect_data.sample2.isin(monos))]
              [['sample1','sample2','combined_effect', 'Count']])

        mono_t.sample1=mono_t.sample1.replace(['Mono'], np.nan)
        mono_t.sample1 = mono_t.sample1.fillna(mono_t.sample2)

        mono_t.sample2=mono_t.sample2.replace(['Mono'], np.nan)
        mono_t.sample2 = mono_t.sample2.fillna(mono_t.sample1)

        mono=pd.DataFrame(mono_t.groupby(['sample1','sample2'],
                                        as_index=False)['combined_effect'].mean())
        mono['Count']=mono_t.groupby(['sample1','sample2'],
                                    as_index=False)['Count'].sum()['Count']           

        mono=mono.rename(columns={'combined_effect':'effect'})

        mono['max_od'] = mono.sample1.map(max_od_dict)

        monos1=mono.add_prefix('mono_s1_')
        effect_data=effect_data.merge(monos1, left_on='sample1',
                                      right_on='mono_s1_sample1').drop('mono_s1_sample1',axis=1)
        monos2=mono.add_prefix('mono_s2_')
        effect_data=effect_data.merge(monos2, left_on='sample2',
                                      right_on='mono_s2_sample1').drop('mono_s2_sample1',axis=1)

        effect_data['strongest'] = effect_data.apply(lambda x: max(x[["mono_s1_effect","mono_s2_effect"]],
                                                                   key=abs), axis=1)

        effect_data['mean'] = effect_data[["mono_s1_effect","mono_s2_effect"]].mean(axis=1)

        effect_data['additive']=(effect_data["mono_s1_effect"]+effect_data["mono_s2_effect"])

        effect_data['OD_W_mean']=(((effect_data["mono_s1_effect"]*effect_data["mono_s1_max_od"])+
                                   (effect_data["mono_s2_effect"]*effect_data["mono_s2_max_od"]))/
                                   (effect_data["mono_s1_max_od"]+effect_data["mono_s2_max_od"]))   
        effect_data['focal']=chip

        effect_data=effect_data.apply(modified_S, axis=1)
        effect_data=effect_data.apply(comm_size, axis=1)
        effect_data=effect_data[effect_data['Count']>2]
        
        k2_effects_dict[df_counter]=effect_data
        
      
        df_counter+=1

In [9]:
merged_effects_dict={}

for start_chip in range(1,repeats):
    temp_effect_df=pd.DataFrame()
        
    for chip in range(start_chip,len(k2_effects_dict)+1,repeats-1):
        temp_effect_df=temp_effect_df.append(k2_effects_dict[chip])

    merged_effects_dict[start_chip]=temp_effect_df.drop_duplicates()



## Calculate RMSE for each bootstrapped dataset

In [10]:
k2_summ_df=pd.DataFrame(columns=['additive','mean', 'OD_W_mean', 'strongest','modified_strongest'])
counter=1
for effects_df in merged_effects_dict.values():
    effects_df=effects_df[effects_df.comm_size=='pair']
    
    for m in ['additive','mean', 'OD_W_mean', 'strongest','modified_strongest']:
        
        effects_df2=effects_df[[m,'combined_effect']].dropna()
        iqr=sp.stats.iqr(effects_df2.combined_effect)

        k2_summ_df.at[counter, m]=(((mean_squared_error(effects_df2[m],effects_df2['combined_effect']))**0.5)/iqr)
        
    counter+=1

In [11]:
# print("Mean\n",k2_summ_df.mean().round(3))
# print("Median\n",k2_summ_df.median().round(3))
# print("STD\n", k2_summ_df.std().round(3))
# print("Coefficient of variation\n",(k2_summ_df.std()/k2_summ_df.mean()).round(3))

k2_sumsum=pd.DataFrame()
# k2_sumsum['original']=(.mean().round(3))
k2_sumsum['mean']=(k2_summ_df.mean().round(3))
k2_sumsum['mean']=(k2_summ_df.mean().round(3))
k2_sumsum['median']=(k2_summ_df.median().round(3))
k2_sumsum['std']=(k2_summ_df.std().round(3))
k2_sumsum['CoV']=((k2_summ_df.std()/k2_summ_df.mean()).round(3))
k2_sumsum

# additive 0.663
# mean 0.464
# OD_W_mean 0.37
# strongest 0.263
# modified_strongest 0.244


,mean,median,std,CoV
additive,0.663,0.663,0.026,0.039
mean,0.478,0.478,0.010,0.021
OD_W_mean,0.383,0.383,0.008,0.021
strongest,0.292,0.292,0.005,0.017
modified_strongest,0.280,0.280,0.004,0.015


## Models by focal

In [12]:
k2_summ_focal_df=pd.DataFrame()
counter=0
for effects_df in merged_effects_dict.values():
    for f in effects_df.focal.unique():
        effects_df2=effects_df[effects_df.focal==f]
        for v in ['additive','mean', 'OD_W_mean', 'strongest','modified_strongest']:
            effects_df3=effects_df2[[v,'combined_effect']].dropna()
            iqr=sp.stats.iqr(effects_df3.combined_effect)

            k2_summ_focal_df.at[counter, f+'_'+v]=(((mean_squared_error(effects_df3[v],
                                                                        effects_df3['combined_effect']))**0.5)/iqr)

    counter+=1    
    

## Models by sign

In [14]:
def qual_mono (row):
    row['qual_monos']=','.join(sorted(list(row[['qual_e_1','qual_e_2']])))
    return (row)

qm_dict={'-,-':'Negative-Negative','+,-':'Postive-Negative','+,+':'Postive-Postive'}

k2_summ_qm_df=pd.DataFrame()
counter=0
for effects_df in merged_effects_dict.values():
    effects_df['qual_e_1'] = np.where(effects_df['mono_s1_effect'] > 0, '+', '-')
    effects_df['qual_e_2'] = np.where(effects_df['mono_s2_effect'] > 0, '+', '-')
    effects_df=effects_df.apply(qual_mono, axis=1)
    
    for qm in effects_df.qual_monos.unique():
        effects_df2=effects_df[effects_df.qual_monos==qm]

        for v in ['additive','mean', 'OD_W_mean', 'strongest','modified_strongest']:
            effects_df3=effects_df2[[v,'combined_effect']].dropna()
            iqr=sp.stats.iqr(effects_df3.combined_effect)

            k2_summ_qm_df.at[counter, qm_dict[qm]+'__'+v]=(((mean_squared_error(effects_df3[v],
                                                                     effects_df3['combined_effect']))**0.5)/iqr)

    counter+=1    
    

In [ ]:
# k2_summ_df.to_csv('Data/Bootstrap_data/k2_summ_df.csv')
# k2_summ_qm_df.to_csv('Data/Bootstrap_data/k2_summ_qm_df.csv')
# k2_summ_focal_df.to_csv('Data/Bootstrap_data/k2_summ_focal_df.csv')

# Trios with E coli Focal

## Setup
### Load and normalize data from all chips

In [15]:
#redefine timepoints without t3 for this experiment
timepoints=['t0','t1','t2']


#Import data
path="Data/kChip_data/k3/"
c1=pd.read_csv(path+'chip1.csv')[["t0", "t1","t2","sample1","sample2","sample3"]].dropna()
c2=pd.read_csv(path+'chip2.csv')[["t0", "t1","t2","sample1","sample2","sample3"]].dropna()
c3=pd.read_csv(path+'chip3.csv')[["t0", "t1","t2","sample1","sample2","sample3"]].dropna()

#label each chip seperately and merge to one dataset
c1['chip']='A'
c2['chip']='B'
c3['chip']='C'
k3=c1.append(c2).append(c3)

#normalize to t0
k3[timepoints]=k3[timepoints].sub(k3['t0'], axis=0)
k3[timepoints]=k3[timepoints].clip(lower=1).round(3)
k3=k3.replace(r_phylo_dict)


In [16]:
def pair_effects(row):
    
    try:
        row['pair_12_effect']=pair_effect_dict["['"+row['sample1']+"', '"
                                               +row['sample2']+"']"]
    except KeyError:
        try:
            row['pair_12_effect']=pair_effect_dict["['"+row['sample2']+"', '"
                                               +row['sample1']+"']"]
        except KeyError: row['pair_12_effect']=np.nan
            
    try:
        row['pair_13_effect']=pair_effect_dict["['"+row['sample1']+"', '"
                                               +row['sample3']+"']"]
    except KeyError:
        try:
            row['pair_13_effect']=pair_effect_dict["['"+row['sample3']+"', '"
                                               +row['sample1']+"']"]
        except KeyError: row['pair_13_effect']=np.nan
            
    try:
        row['pair_23_effect']=pair_effect_dict["['"+row['sample3']+"', '"
                                               +row['sample2']+"']"]
    except KeyError:
        try:
            row['pair_23_effect']=pair_effect_dict["['"+row['sample2']+"', '"
                                               +row['sample3']+"']"]
        except KeyError: row['pair_23_effect']=np.nan
            
    row['additive_p']= row["pair_12_effect"]+row["pair_13_effect"]+row["pair_23_effect"]
    row['mean_p']= mean(row[['pair_12_effect', 'pair_13_effect','pair_23_effect']])
    row['strongest_p']=max(row[['pair_12_effect', 'pair_13_effect','pair_23_effect']], key=abs)
                           
    return row

## Orginzing and calculating data

### Calculating effect by trio- orginal dataset

In [17]:
#Agregate mono growth from mono and mono+blank wells
mono_df=pd.DataFrame(columns=list(k3))
for i in list(itertools.product(['Mono1','Blank1'],repeat=3))[:-1]:
    data=k3[(k3['sample1']==i[0])&(k3['sample2']==i[1])&(k3['sample3']==i[2])]
    mono_df=mono_df.append(data)
    
focal_mono_median=mono_df.t1.median()

k3_wB=k3.copy()

k3_wB.sample1=k3_wB.sample1.replace(['Blank1'], np.nan)
k3_wB.sample2=k3_wB.sample2.replace(['Blank1'], np.nan)
k3_wB.sample3=k3_wB.sample3.replace(['Blank1'], np.nan)

k3_wB.sample1 = k3_wB.sample1.fillna(k3_wB.sample2)
k3_wB.sample1 = k3_wB.sample1.fillna(k3_wB.sample3)

k3_wB.sample2 = k3_wB.sample2.fillna(k3_wB.sample1)
k3_wB.sample2 = k3_wB.sample2.fillna(k3_wB.sample3)

k3_wB.sample3 = k3_wB.sample3.fillna(k3_wB.sample1)
k3_wB.sample3 = k3_wB.sample3.fillna(k3_wB.sample2)

k3_wB[['sample1','sample2','sample3']]=k3_wB[['sample1','sample2',
                                                          'sample3']].fillna('Blank1')

k3_wB_bs=k3_wB.sample(len(k3), replace=False)

k3_effect_data=k3_wB_bs.groupby(['sample1','sample2','sample3'], 
                         as_index=False)['t1'].median()

k3_effect_data['combined_effect']=log(k3_effect_data['t1']/focal_mono_median)

k3_effect_data['Count']=k3_wB_bs.groupby(['sample1', 'sample2','sample3'], 
                                  as_index=False)['t1'].count()['t1']

#add data for each bug on its own
mono=(k3_effect_data[(k3_effect_data.sample1==k3_effect_data.sample2)&
                  (k3_effect_data.sample1==k3_effect_data.sample3)][['sample1','combined_effect', 
                                                               'Count']]
       .rename(columns={'combined_effect':'effect'}))
mono['qual_e'] = np.where(mono['effect'] > 0, '+', '-')
mono['max_od'] = mono.sample1.map(max_od_dict)


monos1=mono.add_prefix('mono_s1_')
k3_effect_data=k3_effect_data.merge(monos1, how='left', left_on='sample1',
                              right_on='mono_s1_sample1').drop('mono_s1_sample1',
                                                               axis=1)

monos2=mono.add_prefix('mono_s2_')
k3_effect_data=k3_effect_data.merge(monos2, how='left', left_on='sample2',
                              right_on='mono_s2_sample1').drop('mono_s2_sample1',
                                                               axis=1)

monos3=mono.add_prefix('mono_s3_')
k3_effect_data=k3_effect_data.merge(monos3, how='left', left_on='sample3',
                              right_on='mono_s3_sample1').drop('mono_s3_sample1',
                                                               axis=1)


k3_effect_data['additive_s']=(k3_effect_data["mono_s1_effect"]+
                            k3_effect_data["mono_s2_effect"]+
                            k3_effect_data["mono_s3_effect"])

k3_effect_data['mean_s']=mean(k3_effect_data[["mono_s1_effect","mono_s2_effect","mono_s3_effect"]],axis=1)

k3_effect_data['strongest_s'] = k3_effect_data.apply(lambda x: max(x[["mono_s1_effect","mono_s2_effect",
                                                                    "mono_s3_effect"]],key=abs), axis=1)

k3_effect_data['OD_W_mean']=(((k3_effect_data["mono_s1_effect"]*k3_effect_data["mono_s1_max_od"])+
                              (k3_effect_data["mono_s2_effect"]*k3_effect_data["mono_s2_max_od"])+
                              (k3_effect_data["mono_s3_effect"]*k3_effect_data["mono_s3_max_od"]))/
                              sum(k3_effect_data[["mono_s1_max_od","mono_s2_max_od","mono_s3_max_od"]],axis=1))


original_k3=k3_effect_data[k3_effect_data['Count']>2]


In [18]:
for r in original_k3.index:
    row=original_k3.loc[r]

    if ((row.sample1!=row.sample2)&
        (row.sample1!=row.sample3)&
        (row.sample2!=row.sample3)&
        (row.sample1!='Mono1')&(row.sample2!='Mono1')&(row.sample3!='Mono1')):

        original_k3.at[r,'comm_size']='trio'

    elif(((row.sample1==row.sample2)&(row.sample1!=row.sample3)&
          (row.sample1!='Mono1')&(row.sample3!='Mono1'))|

         ((row.sample1==row.sample3)&(row.sample1!=row.sample2)&
          (row.sample1!='Mono1')&(row.sample2!='Mono1'))|

         ((row.sample2==row.sample3)&(row.sample2!=row.sample1)&
          (row.sample1!='Mono1')&(row.sample3!='Mono1'))|

         ((row.sample1=='Mono1')&(row.sample2!='Mono1')&
          (row.sample3!='Mono1')&(row.sample2!=row.sample3))|
         ((row.sample2=='Mono1')&(row.sample1!='Mono1')&
          (row.sample3!='Mono1')&(row.sample1!=row.sample3))|
         ((row.sample3=='Mono1')&(row.sample2!='Mono1')&
          (row.sample1!='Mono1')&(row.sample2!=row.sample1))):

        original_k3.at[r,'comm_size']='pair'

    else: original_k3.at[r,'comm_size']='single'

#calculate pair effects
pair_data=original_k3[original_k3.comm_size=='pair']

#merge wells with monos with wells without 
pair_data[['sample1','sample2','sample3',]]=pair_data[['sample1','sample2','sample3',]].replace(['Mono1'], np.nan)

pair_data.sample1 = pair_data.sample1.fillna(pair_data.sample2)
pair_data.sample1 = pair_data.sample1.fillna(pair_data.sample3)

pair_data.sample2 = pair_data.sample2.fillna(pair_data.sample1)
pair_data.sample2 = pair_data.sample2.fillna(pair_data.sample3)

pair_data.sample3 = pair_data.sample3.fillna(pair_data.sample1)
pair_data.sample3 = pair_data.sample3.fillna(pair_data.sample2)

#add column for with pair for each row
pair_data['trio'] =((pair_data[['sample1','sample2','sample3']]).values.tolist())
pair_data['pair'] = pair_data.apply(lambda x: sorted(list(set(x['trio']))),axis=1)
pair_data['pair'] = pair_data.apply(lambda x: (str(x['pair'])),axis=1)
pair_data['trio'] = pair_data.apply(lambda x: (str(x['trio'])),axis=1)

#organize data for comparison between diff starting densities
melted=pair_data.melt(id_vars=['pair'], value_vars=['combined_effect'],)
pair_pairs=pd.DataFrame(columns=['pair','ratio1','ratio2','ratio3'],index=range(len(list(melted.pair.unique()))))

#new dataset with 2:1 and 1:2 ratios in each row
for n,ii in enumerate(list(melted.pair.unique())):
    if len(melted[melted.pair==ii])==3:
        pair_pairs.loc[n]['pair']=ii
        pair_pairs.loc[n]['ratio1']=(list((melted[melted.pair==ii]).value))[0]
        pair_pairs.loc[n]['ratio2']=(list((melted[melted.pair==ii]).value))[1]
        pair_pairs.loc[n]['ratio3']=(list((melted[melted.pair==ii]).value))[2]
    elif len(melted[melted.pair==i])==2:
        pair_pairs.loc[n]['pair']=ii
        pair_pairs.loc[n]['ratio1']=(list((melted[melted.pair==ii]).value))[0]
        pair_pairs.loc[n]['ratio2']=(list((melted[melted.pair==ii]).value))[1]

    #16 pairs only have one ratio, but are included here to use in the dictionary downstream 
    elif len(melted[melted.pair==ii])==1:
        pair_pairs.loc[n]['pair']=i
        pair_pairs.loc[n]['ratio1']=(list((melted[melted.pair==ii]).value))[0]
        pair_pairs.loc[n]['ratio2']=(list((melted[melted.pair==ii]).value))[0]

    else: continue

pair_effect_dict=melted.groupby('pair').mean()['value'].to_dict()

#calculate trio predictoins based on pairs
original_k3=original_k3.apply(pair_effects,axis=1)  


In [19]:
k3_summ_original=pd.DataFrame(columns=['additive_s','mean_s','OD_W_mean','strongest_s',
                                 'additive_p','mean_p','strongest_p'])
counter=0
for v in ['additive_s','mean_s','OD_W_mean','strongest_s',
          'additive_p','mean_p','strongest_p']:
    effects_df=original_k3[original_k3.comm_size=='trio']
    effects_df2=effects_df[[v,'combined_effect']].dropna()
    iqr=sp.stats.iqr(effects_df2.combined_effect)

    k3_summ_original.at[counter, v]=(((mean_squared_error(effects_df2[v],effects_df2['combined_effect']))**0.5)/iqr)
        
counter+=1 

print("Mean")
print(k3_summ_original.mean().round(3))


Mean
additive_s     2.855
mean_s         1.318
OD_W_mean      1.036
strongest_s    0.508
additive_p     4.794
mean_p         0.616
strongest_p    0.418
dtype: float64


### Calculating effect by trio- Bootstrapping

In [20]:
repeats=1001
k3_effects_dict={}

        
#Agregate mono growth from mono and mono+blank wells
mono_df=pd.DataFrame(columns=list(k3))
for i in list(itertools.product(['Mono1','Blank1'],repeat=3))[:-1]:
    data=k3[(k3['sample1']==i[0])&(k3['sample2']==i[1])&(k3['sample3']==i[2])]
    mono_df=mono_df.append(data)
    
focal_mono_median=mono_df.t1.median()

k3_wB=k3.copy()

k3_wB.sample1=k3_wB.sample1.replace(['Blank1'], np.nan)
k3_wB.sample2=k3_wB.sample2.replace(['Blank1'], np.nan)
k3_wB.sample3=k3_wB.sample3.replace(['Blank1'], np.nan)

k3_wB.sample1 = k3_wB.sample1.fillna(k3_wB.sample2)
k3_wB.sample1 = k3_wB.sample1.fillna(k3_wB.sample3)

k3_wB.sample2 = k3_wB.sample2.fillna(k3_wB.sample1)
k3_wB.sample2 = k3_wB.sample2.fillna(k3_wB.sample3)

k3_wB.sample3 = k3_wB.sample3.fillna(k3_wB.sample1)
k3_wB.sample3 = k3_wB.sample3.fillna(k3_wB.sample2)

k3_wB[['sample1','sample2','sample3']]=k3_wB[['sample1','sample2',
                                                          'sample3']].fillna('Blank1')

for i in range(1,repeats):

    k3_wB_bs=k3_wB.sample(len(k3), replace=True, random_state=i)

    k3_effect_data=k3_wB_bs.groupby(['sample1','sample2','sample3'], 
                             as_index=False)['t1'].median()

    k3_effect_data['combined_effect']=log(k3_effect_data['t1']/focal_mono_median)

    k3_effect_data['Count']=k3_wB_bs.groupby(['sample1', 'sample2','sample3'], 
                                      as_index=False)['t1'].count()['t1']

    #add data for each bug on its own
    mono=(k3_effect_data[(k3_effect_data.sample1==k3_effect_data.sample2)&
                      (k3_effect_data.sample1==k3_effect_data.sample3)][['sample1','combined_effect', 
                                                                   'Count']]
           .rename(columns={'combined_effect':'effect'}))
    mono['qual_e'] = np.where(mono['effect'] > 0, '+', '-')
    mono['max_od'] = mono.sample1.map(max_od_dict)


    monos1=mono.add_prefix('mono_s1_')
    k3_effect_data=k3_effect_data.merge(monos1, how='left', left_on='sample1',
                                  right_on='mono_s1_sample1').drop('mono_s1_sample1',
                                                                   axis=1)

    monos2=mono.add_prefix('mono_s2_')
    k3_effect_data=k3_effect_data.merge(monos2, how='left', left_on='sample2',
                                  right_on='mono_s2_sample1').drop('mono_s2_sample1',
                                                                   axis=1)

    monos3=mono.add_prefix('mono_s3_')
    k3_effect_data=k3_effect_data.merge(monos3, how='left', left_on='sample3',
                                  right_on='mono_s3_sample1').drop('mono_s3_sample1',
                                                                   axis=1)


    k3_effect_data['additive_s']=(k3_effect_data["mono_s1_effect"]+
                                k3_effect_data["mono_s2_effect"]+
                                k3_effect_data["mono_s3_effect"])

    k3_effect_data['mean_s']=mean(k3_effect_data[["mono_s1_effect","mono_s2_effect","mono_s3_effect"]],axis=1)

    k3_effect_data['strongest_s'] = k3_effect_data.apply(lambda x: max(x[["mono_s1_effect","mono_s2_effect",
                                                                        "mono_s3_effect"]],key=abs), axis=1)
    
    k3_effect_data['OD_W_mean']=(((k3_effect_data["mono_s1_effect"]*k3_effect_data["mono_s1_max_od"])+
                                  (k3_effect_data["mono_s2_effect"]*k3_effect_data["mono_s2_max_od"])+
                                  (k3_effect_data["mono_s3_effect"]*k3_effect_data["mono_s3_max_od"]))/
                                  sum(k3_effect_data[["mono_s1_max_od","mono_s2_max_od","mono_s3_max_od"]],axis=1))


    k3_effects_dict[i]=k3_effect_data[k3_effect_data['Count']>2]


In [21]:
for i in range(1,repeats):
    #call each bootstrap individually
    temp_data=k3_effects_dict[i]
    
    #add comm_size column
    for r in temp_data.index:
        row=temp_data.loc[r]

        if ((row.sample1!=row.sample2)&
            (row.sample1!=row.sample3)&
            (row.sample2!=row.sample3)&
            (row.sample1!='Mono1')&(row.sample2!='Mono1')&(row.sample3!='Mono1')):

            temp_data.at[r,'comm_size']='trio'

        elif(((row.sample1==row.sample2)&(row.sample1!=row.sample3)&
              (row.sample1!='Mono1')&(row.sample3!='Mono1'))|

             ((row.sample1==row.sample3)&(row.sample1!=row.sample2)&
              (row.sample1!='Mono1')&(row.sample2!='Mono1'))|

             ((row.sample2==row.sample3)&(row.sample2!=row.sample1)&
              (row.sample1!='Mono1')&(row.sample3!='Mono1'))|

             ((row.sample1=='Mono1')&(row.sample2!='Mono1')&
              (row.sample3!='Mono1')&(row.sample2!=row.sample3))|
             ((row.sample2=='Mono1')&(row.sample1!='Mono1')&
              (row.sample3!='Mono1')&(row.sample1!=row.sample3))|
             ((row.sample3=='Mono1')&(row.sample2!='Mono1')&
              (row.sample1!='Mono1')&(row.sample2!=row.sample1))):
                
            temp_data.at[r,'comm_size']='pair'

        else: temp_data.at[r,'comm_size']='single'

    #calculate pair effects
    pair_data=temp_data[temp_data.comm_size=='pair']
    
    #merge wells with monos with wells without 
    pair_data[['sample1','sample2','sample3',]]=pair_data[['sample1','sample2','sample3',]].replace(['Mono1'], np.nan)

    pair_data.sample1 = pair_data.sample1.fillna(pair_data.sample2)
    pair_data.sample1 = pair_data.sample1.fillna(pair_data.sample3)

    pair_data.sample2 = pair_data.sample2.fillna(pair_data.sample1)
    pair_data.sample2 = pair_data.sample2.fillna(pair_data.sample3)

    pair_data.sample3 = pair_data.sample3.fillna(pair_data.sample1)
    pair_data.sample3 = pair_data.sample3.fillna(pair_data.sample2)

    #add column for with pair for each row
    pair_data['trio'] =((pair_data[['sample1','sample2','sample3']]).values.tolist())
    pair_data['pair'] = pair_data.apply(lambda x: sorted(list(set(x['trio']))),axis=1)
    pair_data['pair'] = pair_data.apply(lambda x: (str(x['pair'])),axis=1)
    pair_data['trio'] = pair_data.apply(lambda x: (str(x['trio'])),axis=1)

    #organize data for comparison between diff starting densities
    melted=pair_data.melt(id_vars=['pair'], value_vars=['combined_effect'],)
    pair_pairs=pd.DataFrame(columns=['pair','ratio1','ratio2','ratio3'],index=range(len(list(melted.pair.unique()))))

    #new dataset with 2:1 and 1:2 ratios in each row
    for n,ii in enumerate(list(melted.pair.unique())):
        if len(melted[melted.pair==ii])==3:
            pair_pairs.loc[n]['pair']=ii
            pair_pairs.loc[n]['ratio1']=(list((melted[melted.pair==ii]).value))[0]
            pair_pairs.loc[n]['ratio2']=(list((melted[melted.pair==ii]).value))[1]
            pair_pairs.loc[n]['ratio3']=(list((melted[melted.pair==ii]).value))[2]
        elif len(melted[melted.pair==i])==2:
            pair_pairs.loc[n]['pair']=ii
            pair_pairs.loc[n]['ratio1']=(list((melted[melted.pair==ii]).value))[0]
            pair_pairs.loc[n]['ratio2']=(list((melted[melted.pair==ii]).value))[1]
    
        #16 pairs only have one ratio, but are included here to use in the dictionary downstream 
        elif len(melted[melted.pair==ii])==1:
            pair_pairs.loc[n]['pair']=i
            pair_pairs.loc[n]['ratio1']=(list((melted[melted.pair==ii]).value))[0]
            pair_pairs.loc[n]['ratio2']=(list((melted[melted.pair==ii]).value))[0]

        else: continue

    pair_effect_dict=melted.groupby('pair').mean()['value'].to_dict()
    
    #calculate trio predictoins based on pairs
    k3_effects_dict[i]=temp_data.apply(pair_effects,axis=1)  


In [22]:
k3_summ_df=pd.DataFrame(columns=['additive_s','mean_s','OD_W_mean','strongest_s',
                                 'additive_p','mean_p','strongest_p'])
counter=0
for effects_df in k3_effects_dict.values():
    for v in ['additive_s','mean_s','OD_W_mean','strongest_s',
              'additive_p','mean_p','strongest_p']:
        effects_df=effects_df[effects_df.comm_size=='trio']
        effects_df2=effects_df[[v,'combined_effect']].dropna()
        iqr=sp.stats.iqr(effects_df2.combined_effect)
        
        k3_summ_df.at[counter, v]=(((mean_squared_error(effects_df2[v],effects_df2['combined_effect']))**0.5)/iqr)
        
    counter+=1    

In [23]:
k3_sumsum=pd.DataFrame()
k3_sumsum['original']=(k3_summ_original.mean().round(3))
k3_sumsum['mean']=(k3_summ_df.mean().round(3))
k3_sumsum['mean']=(k3_summ_df.mean().round(3))
k3_sumsum['median']=(k3_summ_df.median().round(3))
k3_sumsum['std']=(k3_summ_df.std().round(3))
k3_sumsum['CoV']=((k3_summ_df.std()/k3_summ_df.mean()).round(3))
k3_sumsum


,original,mean,median,std,CoV
additive_s,2.855,2.714,2.714,0.172,0.063
mean_s,1.318,1.222,1.222,0.055,0.045
OD_W_mean,1.036,0.977,0.977,0.034,0.035
strongest_s,0.508,0.793,0.793,0.065,0.082
additive_p,4.794,4.252,4.252,0.070,0.016
mean_p,0.616,0.674,0.674,0.038,0.056
strongest_p,0.418,0.577,0.577,0.026,0.044


In [24]:
# k3_summ_df.to_csv('Data/Bootstrap_data/k3_summ_df.csv')